In [1]:
# created on Dec 24, 2020
# @author:          Sam Chao
# @email:           zhaobo@uw.edu
# @website:         https://hgis.uw.edu
# @organization:    Department of Geography, University of Washington, Seattle
# @description:     Search geo-tagged tweets within the U.S. This script is modified from https://github.com/shawn-terryah/Twitter_Geolocation

!python -m pip install tweepy
import tweepy, json, time

In [2]:
class StreamListener(tweepy.StreamListener):
    """tweepy.StreamListener is a class provided by tweepy used to access
    the Twitter Streaming API to collect tweets in real-time.
    """

    def __init__(self, time_limit=60, file=""):
        """class initialization"""
        self.start_time = time.time()
        self.limit = time_limit
        self.f = open(file, 'a', encoding="utf-8")
        super(StreamListener, self).__init__()

    def on_data(self, data):
        """This is called when data are streamed in."""
        if (time.time() - self.start_time) < self.limit:
            datajson = json.loads(data)
            print(datajson, "\n")
            if 'id' not in datajson.keys():
                time.sleep(10)
            else:
                # {'limit': {'track': 13, 'timestamp_ms': '1585851016736'}}

                id = datajson['id']
                username = datajson['user']['screen_name']
                created_at = datajson['created_at']
                text = datajson['text'].strip().replace("\n", "")

                # process the geo-tags
                if datajson['coordinates'] == None:
                    try:
                        bbox = datajson['place']['bounding_box']['coordinates'][0]
                        lng = (bbox[0][0] + bbox[2][0]) / 2.0
                        lat = (bbox[0][1] + bbox[1][1]) / 2.0
                    except:
                        lat = 0
                        lng = 0
                else:
                    lng = datajson['coordinates']['coordinates'][0]
                    lat = datajson['coordinates']['coordinates'][1]

                if lat != 0:
                    record = '%s, %s, %f, %f, %s \n' % (username, created_at, lng, lat, text)
                    print(record)
                    self.f.write(record)
                else:
                    pass
        else:
            self.f.close()
            print("finished")
            return False

In [3]:
if __name__ == "__main__":
    # These are provided to you through the Twitter API after you create a account
    # register a Twitter App to get the keys and access tokens.
    output_file = "assets/tweets.csv"

    # Apply for your own Twitter API keys at https://developer.twitter.com/en/apply-for-access
    consumer_key = "fmm3nA45TbARTaKwXcx3D8tMU"
    consumer_secret = "6mQvjRIaOXEp5mvik2a9Nr2mTID2V3Lqy6cOzKS4Xr0FBbskM2"
    access_token = "709921199027986432-FZbrMggRBunEnP6ZQs3VvaEawKRX6i1"
    access_token_secret = "CtXTjDn672EoG1s6CnZAOX7kMe1gF7RTLowI7q4nq3z4i"

    myauth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    myauth.set_access_token(access_token, access_token_secret)

    # LOCATIONS are the longitude, latitude coordinate corners for a box that restricts the
    # geographic area from which you will stream tweets. The first two define the southwest
    # corner of the box and the second two define the northeast corner of the box.
    LOCATIONS = [-124.7771694, 24.520833, -66.947028, 49.384472,  # Contiguous US
                 -164.639405, 58.806859, -144.152365, 71.76871,  # Alaska
                 -160.161542, 18.776344, -154.641396, 22.878623]  # Hawaii

    stream_listener = StreamListener(time_limit=60, file=output_file)
    stream = tweepy.Stream(auth=myauth, listener=stream_listener)
    stream.filter(locations=LOCATIONS)
    stream.filter(track=['asian'], is_async=True)

{'created_at': 'Thu Apr 30 07:00:34 +0000 2020', 'id': 1255753896468770816, 'id_str': '1255753896468770816', 'text': 'https://t.co/q4M25KLCfC', 'display_text_range': [0, 0], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 759876646279053312, 'id_str': '759876646279053312', 'name': 'Mary Maryyy', 'screen_name': 'Mrs_Pops_Leyva', 'location': None, 'url': None, 'description': None, 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 77, 'friends_count': 164, 'listed_count': 0, 'favourites_count': 3680, 'statuses_count': 6012, 'created_at': 'Sun Jul 31 22:21:25 +0000 2016', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_backgrou

{'created_at': 'Thu Apr 30 07:00:35 +0000 2020', 'id': 1255753900445097986, 'id_str': '1255753900445097986', 'text': 'https://t.co/xnlPOxU9UY', 'display_text_range': [0, 0], 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': 1255750272548208641, 'in_reply_to_status_id_str': '1255750272548208641', 'in_reply_to_user_id': 521822818, 'in_reply_to_user_id_str': '521822818', 'in_reply_to_screen_name': 'MaxTrumbull', 'user': {'id': 521822818, 'id_str': '521822818', 'name': 'Maxwell', 'screen_name': 'MaxTrumbull', 'location': 'Toledo, OH', 'url': None, 'description': "UT '20 II ΑΣΦ II", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 614, 'friends_count': 983, 'listed_count': 3, 'favourites_count': 14598, 'statuses_count': 10179, 'created_at': 'Mon Mar 12 00:49:34 +0000 2012', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabl

{'created_at': 'Thu Apr 30 07:00:36 +0000 2020', 'id': 1255753903557226497, 'id_str': '1255753903557226497', 'text': 'Fucking Burke!!!! Omgggg', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 38038743, 'id_str': '38038743', 'name': '\U0001fa90💫 Planet K', 'screen_name': 'Kreative_Kayy', 'location': 'Florida, USA', 'url': 'http://Disney.Is.Life', 'description': 'If I did something to hurt you 🥺, Good Bitch! You probably deserved it. And I’ll do it again 💕', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 1572, 'friends_count': 2264, 'listed_count': 5, 'favourites_count': 33411, 'statuses_count': 51279, 'created_at': 'Tue May 05 22:30:09 +0000 2009', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, '

{'created_at': 'Thu Apr 30 07:00:36 +0000 2020', 'id': 1255753904261742593, 'id_str': '1255753904261742593', 'text': 'D.R.I.\nRepulsion\nSepultura \nThe Germs\nAutopsy', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 2446230549, 'id_str': '2446230549', 'name': 'EraserheaD', 'screen_name': 'Pistol_Ed33', 'location': '𖤐 ⁶⁶⁶ 𖤐', 'url': 'https://www.google.com', 'description': 'Die wunder dieser welt werden dir geschenkt | UCSB', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 297, 'friends_count': 265, 'listed_count': 0, 'favourites_count': 24464, 'statuses_count': 2499, 'created_at': 'Sun Mar 30 04:57:26 +0000 2014', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled

{'created_at': 'Thu Apr 30 07:00:37 +0000 2020', 'id': 1255753907835371520, 'id_str': '1255753907835371520', 'text': 'The skies are empty', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 245119554, 'id_str': '245119554', 'name': 'Zaid', 'screen_name': 'Darth_Zaider67', 'location': 'Nashville', 'url': None, 'description': 'Just livin the dream', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 420, 'friends_count': 358, 'listed_count': 4, 'favourites_count': 2797, 'statuses_count': 7466, 'created_at': 'Sun Jan 30 22:56:34 +0000 2011', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': '131516', 'profile_bac

...

{'created_at': 'Thu Apr 30 07:01:31 +0000 2020', 'id': 1255754136605376512, 'id_str': '1255754136605376512', 'text': '@PaperBugDev YESS', 'display_text_range': [13, 17], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': 1255587890316394500, 'in_reply_to_status_id_str': '1255587890316394500', 'in_reply_to_user_id': 745048814868783104, 'in_reply_to_user_id_str': '745048814868783104', 'in_reply_to_screen_name': 'PaperBugDev', 'user': {'id': 589361024, 'id_str': '589361024', 'name': 'MegaBbones 🦎🎨', 'screen_name': 'megabbones', 'location': 'Springdale, AR', 'url': 'http://youtube.com/user/megabbones', 'description': 'YouTuber ｜Art Education Major ｜young artist ｜positivity ｜ Nintendo lover｜loves you｜ Pikmin｜Warcraft｜Olimar Main｜Switch: SW-6729-3084-8304', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 240, 'friends_count': 2226, 'listed_count': 5, 'favourites_co

{'created_at': 'Thu Apr 30 07:01:33 +0000 2020', 'id': 1255754143186259968, 'id_str': '1255754143186259968', 'text': "@mvbrat91 @PrinceArthasLK What in F's name are you talking about", 'display_text_range': [26, 64], 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>', 'truncated': False, 'in_reply_to_status_id': 1255752540546875392, 'in_reply_to_status_id_str': '1255752540546875392', 'in_reply_to_user_id': 1158666440, 'in_reply_to_user_id_str': '1158666440', 'in_reply_to_screen_name': 'mvbrat91', 'user': {'id': 1076852586538520576, 'id_str': '1076852586538520576', 'name': 'Maureen And Michael🆘️', 'screen_name': 'MaureenPiscate7', 'location': 'Massachusetts, USA', 'url': 'http://ManorHQ.com', 'description': "24 years together. I'm Atheist, she's Wiccan. Writer. Musician. TheResistance. #Wrestling #Horror \nColumnist  for @OfficialManor. Cats rule. Homeless advocate.", 'translator_type': 'none', 'protected': False, 'verified': False, 'followe

{'created_at': 'Thu Apr 30 07:01:33 +0000 2020', 'id': 1255754145501503488, 'id_str': '1255754145501503488', 'text': 'Can’t quit 💯', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'truncated': False, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 1165291278, 'id_str': '1165291278', 'name': 'soy', 'screen_name': 'marquisjjones', 'location': 'Dallas, TX', 'url': None, 'description': 'Laugh mas.', 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 785, 'friends_count': 1176, 'listed_count': 3, 'favourites_count': 15478, 'statuses_count': 25202, 'created_at': 'Sun Feb 10 08:22:11 +0000 2013', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': None, 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED', 'profile_background_image

{'created_at': 'Thu Apr 30 07:01:33 +0000 2020', 'id': 1255754146327793664, 'id_str': '1255754146327793664', 'text': 'On the red shores of #carcosa ...\nAchievement Unlocked: Cheezy 1980’s Horror Poster look. I think I can call this o… https://t.co/mGonVx6ap7', 'source': '<a href="http://instagram.com" rel="nofollow">Instagram</a>', 'truncated': True, 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 26166753, 'id_str': '26166753', 'name': 'Mani Price @ DarksideoftheCon', 'screen_name': 'ManiTheUncanny', 'location': 'Instagram: ManiTheUncanny', 'url': 'http://manitheuncanny.com/about/', 'description': "I'm Mani Price, a visionary artist & tarot reader. My work focuses on women’s sexual empowerment, symbolism & magick. I do commissions for all spiritual needs.", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 667, 'friends_cou

finished
